In [5]:
import ir_datasets
import pandas as pd
import re
import json
import os
import html
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

# تحميل موارد NLTK
nltk.download('stopwords')

# 1. تحميل مجموعة البيانات
dataset = ir_datasets.load("msmarco-passage/train")

# 2. قراءة الوثائق وتحويلها إلى قائمة
docs_data = []
count = 0
LIMIT = 500_000
seen_ids = set()  # لمنع التكرار

# 3. تنظيف Unicode المشوه
def safe_text(text):
    try:
        text = html.unescape(text)  # إزالة HTML entities مثل &amp;
        text = unicodedata.normalize("NFKD", text)  # Normalize Unicode
        text = ''.join(c for c in text if c.isprintable())  # إزالة الأحرف غير القابلة للطباعة
        return text
    except:
        return text

# 4. أدوات المعالجة اللغوية
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

# 5. دالة التنظيف المتقدمة – تُرجع قائمة كلمات (tokens)
def advanced_preprocess(text):
    # Lowercase
    text = text.lower()
    
    # إزالة الروابط
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # إزالة الإيميلات
    text = re.sub(r'\S+@\S+', '', text)

    # إزالة علامات HTML
    text = re.sub(r'<.*?>', '', text)

    # إزالة الأرقام
    text = re.sub(r'\d+', '', text)

    # إزالة علامات الترقيم والرموز الخاصة
    text = re.sub(r'[^a-z\s]', ' ', text)

    # إزالة التكرار الزائد في الحروف
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # إزالة الفراغات الزائدة
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization + Stopword Removal + Stemming
    tokens = text.split()
    processed = [stemmer.stem(word) for word in tokens if word not in stop_words and len(word) > 2]

    return processed

# 6. تحميل وتنظيف الوثائق
doc_iterator = dataset.docs_iter()

while count < LIMIT:
    try:
        doc = next(doc_iterator)

        if doc.doc_id in seen_ids:
            continue
        seen_ids.add(doc.doc_id)

        clean_original = safe_text(doc.text)

        docs_data.append({
            "id": doc.doc_id,
            "text": clean_original
        })
        count += 1

        if count % 10000 == 0:
            print(f"✅ تم تحميل {count} وثيقة...")

    except Exception as e:
        continue

# 7. تحويل إلى DataFrame
df = pd.DataFrame(docs_data)

# 8. تطبيق التنظيف المتقدم => tokens
print("\n🧹 بدء تنظيف النصوص وتحويلها إلى Tokens...")
df['tokens'] = df['text'].apply(advanced_preprocess)

# 9. إزالة التكرارات حسب ID
df = df.drop_duplicates(subset='id', keep='first')

# 10. حفظ النتائج على شكل JSON
output_dir = r"..\data\msmarco_train_test\processed"
output_file = os.path.join(output_dir, "tokenized_msmarco.json")
os.makedirs(output_dir, exist_ok=True)

df[['id', 'tokens']].to_json(output_file, orient='records', lines=False, force_ascii=False, indent=2)


print(f"\n📁 تم حفظ البيانات بنجاح في الملف:\n{output_file}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ تم تحميل 10000 وثيقة...
✅ تم تحميل 20000 وثيقة...
✅ تم تحميل 30000 وثيقة...
✅ تم تحميل 40000 وثيقة...
✅ تم تحميل 50000 وثيقة...
✅ تم تحميل 60000 وثيقة...
✅ تم تحميل 70000 وثيقة...
✅ تم تحميل 80000 وثيقة...
✅ تم تحميل 90000 وثيقة...
✅ تم تحميل 100000 وثيقة...
✅ تم تحميل 110000 وثيقة...
✅ تم تحميل 120000 وثيقة...
✅ تم تحميل 130000 وثيقة...
✅ تم تحميل 140000 وثيقة...
✅ تم تحميل 150000 وثيقة...
✅ تم تحميل 160000 وثيقة...
✅ تم تحميل 170000 وثيقة...
✅ تم تحميل 180000 وثيقة...
✅ تم تحميل 190000 وثيقة...
✅ تم تحميل 200000 وثيقة...
✅ تم تحميل 210000 وثيقة...
✅ تم تحميل 220000 وثيقة...
✅ تم تحميل 230000 وثيقة...
✅ تم تحميل 240000 وثيقة...
✅ تم تحميل 250000 وثيقة...
✅ تم تحميل 260000 وثيقة...
✅ تم تحميل 270000 وثيقة...
✅ تم تحميل 280000 وثيقة...
✅ تم تحميل 290000 وثيقة...
✅ تم تحميل 300000 وثيقة...
✅ تم تحميل 310000 وثيقة...
✅ تم تحميل 320000 وثيقة...
✅ تم تحميل 330000 وثيقة...
✅ تم تحميل 340000 وثيقة...
✅ تم تحميل 350000 وثيقة...
✅ تم تحميل 360000 وثيقة...
✅ تم تحميل 370000 وثيقة...
✅ تم تحميل

In [6]:
import json
import os
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import re

sys.path.append("..")

from services.documents_service import advanced_preprocess

# 1. تحميل البيانات
input_file = r"..\data\msmarco_train_test\processed\tokenized_msmarco.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. استخراج النصوص والمعرفات
doc_ids = [doc['id'] for doc in data]
texts = [" ".join(doc['tokens']) for doc in data]  # نصوص نصية

# 3. إعداد TfidfVectorizer مع preprocess كـ tokenizer
vectorizer = TfidfVectorizer(
    tokenizer=advanced_preprocess,   # دالة تقسيم الكلمات مع التنظيف
    preprocessor=None,      # لا تستخدم preprocessor آخر
    max_df=0.85,
    min_df=2,
    sublinear_tf=True,
    norm='l2',
    lowercase=False         # لأن preprocess تقوم بتحويل الحروف إلى صغيرة بالفعل
)

# 4. تطبيق TF-IDF
tfidf_matrix = vectorizer.fit_transform(texts)

# 5. حفظ النتائج
output_dir = r"..\data\msmarco_train_test\index\TFIDF"
os.makedirs(output_dir, exist_ok=True)

joblib.dump(vectorizer, os.path.join(output_dir, "tfidf_vectorizer_msmarco_train_test.joblib"))
joblib.dump(tfidf_matrix, os.path.join(output_dir, "tfidf_matrix_msmarco_train_test.joblib"))
joblib.dump(doc_ids, os.path.join(output_dir, "doc_ids_msmarco_train_test.joblib"))

print("✅ تم حفظ ملفات TF-IDF بنجاح.")


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


✅ تم حفظ ملفات TF-IDF بنجاح.


In [7]:
# ------------------ التحميلات الأساسية ------------------
import os
import re
import html
import joblib
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

import sys
sys.path.append("..")

from services.documents_service import advanced_preprocess

# ------------------ 1. تحديد المسارات ------------------
print("--- 1. تحديد مسارات الملفات ---")
doc_ids_path = r"..\data\msmarco_train_test\index\TFIDF\doc_ids_msmarco_train_test.joblib"
tfidf_matrix_path = r"..\data\msmarco_train_test\index\TFIDF\tfidf_matrix_msmarco_train_test.joblib"
tfidf_vectorizer_path = r"..\data\msmarco_train_test\index\TFIDF\tfidf_vectorizer_msmarco_train_test.joblib"

output_inverted_index_path = r"..\data\msmarco_train_test\index\TFIDF\tfidf_inverted_index.joblib"

output_dir = os.path.dirname(output_inverted_index_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"تم إنشاء مجلد الإخراج: {output_dir}")
else:
    print(f"مجلد الإخراج موجود: {output_dir}")

# ------------------ 2. تحميل الملفات ------------------
print("\n--- 2. جاري تحميل الملفات ---")
try:
    doc_ids = joblib.load(doc_ids_path)
    tfidf_matrix = joblib.load(tfidf_matrix_path)
    tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
    print("تم تحميل الملفات بنجاح.")
    print(f"عدد المستندات: {len(doc_ids)}")
    print(f"أبعاد مصفوفة TF-IDF: {tfidf_matrix.shape}")
except FileNotFoundError as e:
    print(f"خطأ: لم يتم العثور على أحد الملفات. يرجى التحقق من المسارات.")
    print(f"المسار المفقود: {e.filename}")
    exit()
except Exception as e:
    print(f"حدث خطأ أثناء تحميل الملفات: {e}")
    exit()

# ------------------ 3. بناء الفهرس المعكوس ------------------
print("\n--- 3. جاري بناء الفهرس المعكوس ---")

feature_names = tfidf_vectorizer.get_feature_names_out()
inverted_index = {}

tfidf_matrix_coo = tfidf_matrix.tocoo()
print("بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...")

for doc_idx, term_idx, tfidf_score in tqdm(zip(tfidf_matrix_coo.row, tfidf_matrix_coo.col, tfidf_matrix_coo.data),
                                           total=len(tfidf_matrix_coo.data),
                                           desc="بناء الفهرس المعكوس"):
    term = feature_names[term_idx]
    doc_id = doc_ids[doc_idx]
    if term not in inverted_index:
        inverted_index[term] = []
    inverted_index[term].append((doc_id, float(tfidf_score)))

print("جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...")
for term in inverted_index:
    inverted_index[term].sort(key=lambda x: x[1], reverse=True)

print("تم بناء الفهرس المعكوس بنجاح.")
print(f"عدد الكلمات الفريدة في الفهرس المعكوس: {len(inverted_index)}")

# ------------------ 4. إعداد البيانات للحفظ ------------------
print("\n--- 4. إعداد البيانات للحفظ ---")
inverted_index_data = {
    "inverted_index": inverted_index,
    "num_documents": len(doc_ids),
    "num_terms": len(inverted_index),
    "vocabulary_size": len(feature_names),
    "vectorizer_vocabulary": dict(tfidf_vectorizer.vocabulary_)
}

# ------------------ 5. حفظ الفهرس ------------------
print("\n--- 5. جاري حفظ الفهرس المعكوس ---")
try:
    joblib.dump(inverted_index_data, output_inverted_index_path)
    print(f"✅ تم حفظ الفهرس المعكوس بنجاح في: {output_inverted_index_path}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء حفظ الفهرس المعكوس: {e}")

# ------------------ 6. التحقق من الحفظ ------------------
print("\n--- 6. التحقق من الفهرس المحفوظ ---")
try:
    loaded_data = joblib.load(output_inverted_index_path)
    loaded_index = loaded_data["inverted_index"]
    print("✅ تم تحميل الفهرس للتحقق.")
    print(f"عدد الكلمات في الفهرس: {len(loaded_index)}")

    example_term = next(iter(loaded_index))
    print(f"مثال على إدخال في الفهرس المعكوس ('{example_term}'): {loaded_index[example_term][:5]}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء التحقق: {e}")

print("\n--- ✅ انتهى تنفيذ الكود بنجاح ---")

--- 1. تحديد مسارات الملفات ---
مجلد الإخراج موجود: ..\data\msmarco_train_test\index\TFIDF

--- 2. جاري تحميل الملفات ---
تم تحميل الملفات بنجاح.
عدد المستندات: 500000
أبعاد مصفوفة TF-IDF: (500000, 91371)

--- 3. جاري بناء الفهرس المعكوس ---
بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...


بناء الفهرس المعكوس:   0%|          | 0/11624617 [00:00<?, ?it/s]

جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...
تم بناء الفهرس المعكوس بنجاح.
عدد الكلمات الفريدة في الفهرس المعكوس: 91371

--- 4. إعداد البيانات للحفظ ---

--- 5. جاري حفظ الفهرس المعكوس ---
✅ تم حفظ الفهرس المعكوس بنجاح في: ..\data\msmarco_train_test\index\TFIDF\tfidf_inverted_index.joblib

--- 6. التحقق من الفهرس المحفوظ ---
✅ تم تحميل الفهرس للتحقق.
عدد الكلمات في الفهرس: 91371
مثال على إدخال في الفهرس المعكوس ('presenc'): [('409792', 0.4636566116093226), ('580660', 0.44391864539155124), ('774505', 0.43167140720627645), ('569503', 0.41676428533006127), ('585454', 0.3902043674130837)]

--- ✅ انتهى تنفيذ الكود بنجاح ---


In [8]:
import pandas as pd
import ir_datasets

dataset = ir_datasets.load("msmarco-passage/train")

seen_ids = set()

doc_iterator = dataset.docs_iter()

count = 0

docs_data = []

def safe_text(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

while count < 500000:

    try:
        doc = next(doc_iterator)

        # تجاهل التكرارات
        if doc.doc_id in seen_ids:
            continue
        seen_ids.add(doc.doc_id)

        text_clean = safe_text(doc.text)

        docs_data.append({
            "id": doc.doc_id,
            "text": text_clean
        })
        count += 1

        if count % 10000 == 0:
            print(f"✅ تم تحميل {count} وثيقة...")

    except Exception:
        continue

# 6. إنشاء DataFrame
df = pd.DataFrame(docs_data)

df.drop(df.loc[df['id']==''].index,inplace=True)

df['id']=df['id'].astype(str)

✅ تم تحميل 10000 وثيقة...
✅ تم تحميل 20000 وثيقة...
✅ تم تحميل 30000 وثيقة...
✅ تم تحميل 40000 وثيقة...
✅ تم تحميل 50000 وثيقة...
✅ تم تحميل 60000 وثيقة...
✅ تم تحميل 70000 وثيقة...
✅ تم تحميل 80000 وثيقة...
✅ تم تحميل 90000 وثيقة...
✅ تم تحميل 100000 وثيقة...
✅ تم تحميل 110000 وثيقة...
✅ تم تحميل 120000 وثيقة...
✅ تم تحميل 130000 وثيقة...
✅ تم تحميل 140000 وثيقة...
✅ تم تحميل 150000 وثيقة...
✅ تم تحميل 160000 وثيقة...
✅ تم تحميل 170000 وثيقة...
✅ تم تحميل 180000 وثيقة...
✅ تم تحميل 190000 وثيقة...
✅ تم تحميل 200000 وثيقة...
✅ تم تحميل 210000 وثيقة...
✅ تم تحميل 220000 وثيقة...
✅ تم تحميل 230000 وثيقة...
✅ تم تحميل 240000 وثيقة...
✅ تم تحميل 250000 وثيقة...
✅ تم تحميل 260000 وثيقة...
✅ تم تحميل 270000 وثيقة...
✅ تم تحميل 280000 وثيقة...
✅ تم تحميل 290000 وثيقة...
✅ تم تحميل 300000 وثيقة...
✅ تم تحميل 310000 وثيقة...
✅ تم تحميل 320000 وثيقة...
✅ تم تحميل 330000 وثيقة...
✅ تم تحميل 340000 وثيقة...
✅ تم تحميل 350000 وثيقة...
✅ تم تحميل 360000 وثيقة...
✅ تم تحميل 370000 وثيقة...
✅ تم تحميل

In [9]:
import os
output_dir = r"..\data\msmarco_train_test\raw"
output_file = os.path.join(output_dir, "raw_msmarco_train_test.json")
os.makedirs(output_dir, exist_ok=True)

df.to_json(output_file, orient='records', lines=True, force_ascii=False)

In [ ]:
# =============================================
# 📦 المكتبات المطلوبة
# =============================================
import ir_datasets
import numpy as np
import os
import joblib
import json
import re
import html
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from trectools import TrecQrel, TrecRun, TrecEval 
from tabulate import tabulate
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from joblib import Memory
import sys
sys.path.append("..")

# =============================================
# ⚙️ تهيئة الكاش
# =============================================
memory = Memory(location='./cache', verbose=0)

# =============================================
# ⚙️ تحميل بيانات ir_datasets (MSMARCO)
# =============================================
dataset = ir_datasets.load("msmarco-passage/train")
queries_path = os.path.expanduser("~/.ir_datasets/msmarco-passage/train/queries.tsv")

queries = {}
with open(queries_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 2:
            queries[parts[0]] = parts[1]

qrels = {}
for qrel in dataset.qrels_iter():
    if qrel.relevance > 0:
        qrels.setdefault(qrel.query_id, set()).add(qrel.doc_id)

# =============================================
# 🧼 دالة التنظيف
# =============================================
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    text = html.unescape(text)
    text = ''.join(c for c in text if c.isprintable())
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return ' '.join(words)

# =============================================
# تحميل ملفات التمثيلات
# =============================================
tfidf_doc_ids = joblib.load(r"../data/msmarco_train_test/index/TFIDF/doc_ids_msmarco_train_test.joblib")
tfidf_matrix = joblib.load(r"../data/msmarco_train_test/index/TFIDF/tfidf_matrix_msmarco_train_test.joblib")
tfidf_vectorizer = joblib.load(r"../data/msmarco_train_test/index/TFIDF/tfidf_vectorizer_msmarco_train_test.joblib")
inverted_index_data = joblib.load(r"../data/msmarco_train_test/index/TFIDF/tfidf_inverted_index.joblib")

bert_embeddings = np.load(r"../data/msmarco_train_test/index/bert/bert_embeddings.npy")
bert_doc_ids = joblib.load(r"../data/msmarco_train_test/index/bert/doc_ids.joblib")
bert_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

docs_dict = {}
with open(r"../data/msmarco_train_test/raw/raw_msmarco_train_test.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            j = json.loads(line)
            docs_dict[str(j["id"])] = j["text"]
        except:
            continue

# =============================================
# ✳️ فلترة qrels و queries
# =============================================
available_doc_ids = set(docs_dict.keys())
filtered_qrels = {
    qid: {docid for docid in docids if docid in available_doc_ids}
    for qid, docids in qrels.items()
}
filtered_qrels = {qid: docids for qid, docids in filtered_qrels.items() if docids}
filtered_queries = {qid: queries[qid] for qid in filtered_qrels}

qrels = filtered_qrels
queries = filtered_queries

# أخذ أول 5000 استعلام فقط
sample_queries = dict(list(queries.items())[:10000])

# =============================================
# 🔍 دوال البحث الأصلية
# =============================================
def search_tfidf_with_inverted_index(query, inverted_index_data, tfidf_vectorizer, tfidf_matrix, doc_ids, docs_dict, top_k=10, candidate_size=100):
    cleaned_query = advanced_preprocess(query)
    query_terms = cleaned_query.split()
    if not query_terms:
        return []

    doc_scores = {}
    for term in query_terms:
        if term in inverted_index_data["inverted_index"]:
            postings = inverted_index_data["inverted_index"][term]
            for doc_id, score in postings:
                doc_scores[doc_id] = doc_scores.get(doc_id, 0) + score

    candidate_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:candidate_size]
    doc_id_to_index = {doc_id: idx for idx, doc_id in enumerate(doc_ids)}
    candidate_indices = [doc_id_to_index[doc_id] for doc_id, _ in candidate_docs if doc_id in doc_id_to_index]

    if not candidate_indices:
        return []

    candidate_tfidf_matrix = tfidf_matrix[candidate_indices]
    query_vector = tfidf_vectorizer.transform([cleaned_query])
    cosine_scores = cosine_similarity(query_vector, candidate_tfidf_matrix).flatten()
    top_indices = cosine_scores.argsort()[::-1][:top_k]

    results = []
    for idx in top_indices:
        doc_idx = candidate_indices[idx]
        doc_id = doc_ids[doc_idx]
        doc_text = docs_dict.get(doc_id, "")
        score = cosine_scores[idx]
        results.append((doc_id, doc_text, score))
    return results

def search_bert(query, top_k=10):
    query_embedding = bert_model.encode([query])
    bert_scores = cosine_similarity(query_embedding, bert_embeddings).flatten()
    top_indices = np.argsort(bert_scores)[::-1][:top_k]
    results = [(bert_doc_ids[i], docs_dict.get(bert_doc_ids[i], ""), bert_scores[i]) for i in top_indices]
    return results

def search_hybrid(query, tfidf_weight=0.5, bert_weight=0.5, top_k=10):
    tfidf_scores = cosine_similarity(tfidf_vectorizer.transform([advanced_preprocess(query)]), tfidf_matrix).flatten()
    bert_scores = cosine_similarity(bert_model.encode([query]), bert_embeddings).flatten()
    combined_scores = tfidf_weight * tfidf_scores + bert_weight * bert_scores
    top_indices = np.argsort(combined_scores)[::-1][:top_k]
    results = [(tfidf_doc_ids[i], docs_dict.get(tfidf_doc_ids[i], ""), combined_scores[i]) for i in top_indices]
    return results

# =============================================
# 🧠 تغليف بالكاش
# =============================================
@memory.cache
def cached_search_tfidf(query, top_k=10, candidate_size=100):
    return search_tfidf_with_inverted_index(query, inverted_index_data, tfidf_vectorizer, tfidf_matrix, tfidf_doc_ids, docs_dict, top_k, candidate_size)

@memory.cache
def cached_search_bert(query, top_k=10):
    return search_bert(query, top_k)

@memory.cache
def cached_search_hybrid(query, tfidf_weight=0.5, bert_weight=0.5, top_k=10):
    return search_hybrid(query, tfidf_weight, bert_weight, top_k)

# =============================================
# 📁 دوال كتابة run و qrel
# =============================================
def write_qrel_file(qrels, filepath):
    with open(filepath, "w") as f:
        for qid, docids in qrels.items():
            for docid in docids:
                f.write(f"{qid} 0 {docid} 1\n")

def write_run_file_threaded(search_fn, queries, run_name, filepath, top_k=10, max_workers=8):
    with open(filepath, "w") as f:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {
                executor.submit(search_fn, query, top_k=top_k): qid
                for qid, query in queries.items()
            }
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Running {run_name}"):
                qid = futures[future]
                try:
                    results = future.result()
                    for rank, (doc_id, _, score) in enumerate(results, start=1):
                        f.write(f"{qid} Q0 {doc_id} {rank} {score} {run_name}\n")
                except Exception as e:
                    print(f"⚠️ Error in query {qid}: {e}")

# =============================================
# 📈 التقييم
# =============================================
qrel_path = "filtered_msmarco.qrel"
run_tfidf_path = "run_tfidf.txt"
run_bert_path = "run_bert.txt"
run_hybrid_path = "run_hybrid.txt"

write_qrel_file(qrels, qrel_path)
write_run_file_threaded(cached_search_tfidf, sample_queries, "TFIDF", run_tfidf_path, top_k=10)
write_run_file_threaded(cached_search_bert, sample_queries, "BERT", run_bert_path, top_k=10)
write_run_file_threaded(lambda q, top_k=10: cached_search_hybrid(q, tfidf_weight=0.4, bert_weight=0.6, top_k=top_k), sample_queries, "Hybrid", run_hybrid_path, top_k=10)

qrel = TrecQrel(qrel_path)
runs = {
    "TFIDF": TrecRun(run_tfidf_path),
    "BERT": TrecRun(run_bert_path),
    "Hybrid": TrecRun(run_hybrid_path),
}

results_table = []

for model_name, run in runs.items():
    evaluation = TrecEval(run, qrel)
    model_results = {
        "Model": model_name,
        "MAP": evaluation.get_map(),
        "MRR": evaluation.get_reciprocal_rank(),
        "P@10": evaluation.get_precision(10),
        "Recall": evaluation.get_recall(1000)
    }
    results_table.append(model_results)

print("\n📊 Evaluation Results:")
print(tabulate(results_table, headers="keys", tablefmt="fancy_grid", floatfmt=".4f"))

for path in [qrel_path, run_tfidf_path, run_bert_path, run_hybrid_path]:
    try:
        os.remove(path)
    except OSError as e:
        print(f"⚠️ فشل حذف {path}: {e}")


Running TFIDF:   0%|                                                                 | 5/10000 [00:00<04:23, 37.90it/s]C:\ProgramData\anaconda3\Lib\site-packages\joblib\memory.py:577: UserWarning: Persisting input arguments took 0.59s to run.If this happens often in your code, it can cause performance problems (results will be correct in all cases). The reason for this is probably some large input arguments for a wrapped function.
  return self._cached_call(args, kwargs, shelving=False)[0]
C:\ProgramData\anaconda3\Lib\site-packages\joblib\memory.py:577: UserWarning: Persisting input arguments took 0.65s to run.If this happens often in your code, it can cause performance problems (results will be correct in all cases). The reason for this is probably some large input arguments for a wrapped function.
  return self._cached_call(args, kwargs, shelving=False)[0]
Running TFIDF:   0%|                                                                | 16/10000 [00:02<25:05,  6.63it/s]C:\Program